In [ ]:
import requests

#common functions

#function to fetch data from given url
def FetchEPLDataFromUrl(urlPath, header):
    try:
        #add content header as text/csv for incoming data
        file = requests.get(urlPath, headers = header)
        return file.text
    except Exception as ex:
        print('Error fetching data from -', urlPath, 'Exception message -', ex)
        return False

#function to save downloaded data in given format
def SaveAsFile(content, fileName):
    try:
        file = open(fileName, 'w')
        file.write(content)
        file.close()
    except Exception as ex:
        print('Error saving file -', fileName, ' content - ', content, 'Exception message -', ex)

In [ ]:
#season ranging from 2015-2024
seasons = list(range(23,26))
baseUrl = 'https://www.football-data.co.uk/mmz4281/'

for i, val in enumerate(seasons):
    if (i + 1) < len(seasons):
        #fetch per season data from specified url
        urlPath = baseUrl + str(val) + str(seasons[i + 1]) + '/E0.csv'
        dataContent = FetchEPLDataFromUrl(urlPath, {'content-type' : 'text/csv'})

        #save data per season as csv file only if valid
        if dataContent != False:
            fileName = 'season_' + str(val) + '-' + str(seasons[i + 1])  + '.csv'
            SaveAsFile(dataContent, fileName)

In [ ]:
#imported glob package to fetch list of all .csv files in current working directory
import glob

files = glob.glob('*.csv')
print('List of csv files in current working directory -')
files

List of csv files in current working directory -


['season_24-25.csv', 'season_23-24.csv']

In [ ]:
!pip install --upgrade tensorflow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 615.3/615.3 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 76.6 MB/s eta 0:00:00
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.17.1
    Uninstalling tensorboard-2.17.1:
      Successfully uninstalled tensorboard-2.17.1
  Attempting uninstall: tensorflow
    Found existing installation: tensorflow 2.17.1
    Uninstalling tensorflow-2.17.1:
      Successfully uninstalled tensorflow-2.17.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tf-keras 2.17.0 requires tensorflow<2.18,>=2.17, but you have tensorflow 2.18.0 which is incompatible.


In [ ]:
!pip install pybettor



In [ ]:
import pandas as pd
from pybettor import implied_prob
import tensorflow as tf
import numpy as np
# Read and combine data
df1 = pd.read_csv(files[0])
df2 = pd.read_csv(files[1])
df = pd.concat([df1, df2], ignore_index=True)

# Extract odds and match data
bet_odds = df.loc[:, 'B365H':'WHA']
market_odds = df.loc[:, 'AvgH':'AvgA']
bet_odds['date'] = df['Date']
market_odds['date'] = df['Date']
bet_odds = pd.concat([bet_odds, market_odds],axis=1)
bet_odds.set_index('date', inplace=True)
# Prepare match data
bet_data = df.loc[:, 'Date':'FTR']

# Process columns in groups of three
for i in range(0, len(bet_odds.columns), 3):  # Skip 'date' column
    col1, col2, col3 = bet_odds.columns[i:i+3]
    new_col = col1[:-1] + '_odds'  # Generate new column name
    bet_data[new_col] = bet_odds[[col1, col2, col3]].values.tolist()
bet_data['total_goals'] = bet_data['FTHG'] + bet_data['FTAG']
# Define a function to convert FTR to one-hot encoding
def encode_ftr(ftr_value):
    if ftr_value == 'H':
        return [1, 0, 0]  # Home win
    elif ftr_value == 'D':
        return [0, 1, 0]  # Draw
    elif ftr_value == 'A':
        return [0, 0, 1]  # Away win
    return [0, 0, 0]  # Default case (in case of missing/invalid values)

# Apply the function to the 'FTR' column
bet_data['FTR'] = bet_data['FTR'].apply(encode_ftr)
for col in bet_data.columns[7:-1]:
  new_column = []
  losses=[]
  for i in range(0,len(bet_data)):
    if not pd.isna(bet_data.loc[i][col]).all():
      implied_probabilities = implied_prob(bet_data.loc[i][col], category="dec")

      # Ensure the predicted values are probabilities between 0 and 1
      predicted_probs = np.array(implied_probabilities)
      true_label = np.array(bet_data.loc[i]['FTR'])

      # Create an instance of the BinaryCrossentropy loss function
      loss_fn = tf.keras.losses.CategoricalCrossentropy(from_logits=False)

      # Calculate the loss (use a batch of one, reshaped)
      loss_value = loss_fn(true_label.reshape(1, -1), predicted_probs.reshape(1, -1))

      # Append the loss to the losses list
      losses.append(loss_value.numpy())
      new_column.append(implied_probabilities)

  bet_data[f'{col}_loss']=pd.Series(losses)
  new_column = pd.Series(new_column, name=col)
  bet_data.update(new_column)
bet_data

,Date,Time,HomeTeam,AwayTeam,FTHG,FTAG,FTR,B365_odds,BW_odds,BF_odds,PS_odds,WH_odds,Avg_odds,total_goals,B365_odds_loss,BW_odds_loss,BF_odds_loss,PS_odds_loss,WH_odds_loss,Avg_odds_loss
0,16/08/2024,20:00,Man United,Fulham,1,0,"[1, 0, 0]","[0.625, 0.23809523809523808, 0.19047619047619047]","[0.625, 0.22727272727272727, 0.19047619047619047]","[0.625, 0.23094688221709006, 0.2]","[0.6134969325153374, 0.22831050228310504, 0.18...","[0.6060606060606061, 0.23809523809523808, 0.2]","[0.6172839506172839, 0.2293577981651376, 0.194...",1,0.522189,0.511864,0.524442,0.518611,0.543984,0.522432
1,17/08/2024,12:30,Ipswich,Liverpool,0,2,"[0, 0, 1]","[0.11764705882352941, 0.18181818181818182, 0.7...","[0.13333333333333333, 0.18181818181818182, 0.7...","[0.11764705882352941, 0.18181818181818182, 0.7...","[0.12224938875305624, 0.17123287671232876, 0.7...","[0.11764705882352941, 0.18181818181818182, 0.7...","[0.12077294685990339, 0.1736111111111111, 0.74...",2,0.335249,0.356699,0.335249,0.331651,0.335249,0.332518
2,17/08/2024,15:00,Arsenal,Wolves,2,0,"[1, 0, 0]","[0.8474576271186441, 0.13333333333333333, 0.07...","[0.8333333333333334, 0.13333333333333333, 0.07...","[0.8695652173913044, 0.125, 0.0625]","[0.8620689655172414, 0.11682242990654206, 0.06...","[0.8474576271186441, 0.14285714285714285, 0.05...","[0.8474576271186441, 0.1272264631043257, 0.063...",2,0.221624,0.222254,0.195258,0.188163,0.213484,0.202517
3,17/08/2024,15:00,Everton,Brighton,0,3,"[0, 0, 1]","[0.3802281368821293, 0.30303030303030304, 0.38...","[0.37735849056603776, 0.29411764705882354, 0.3...","[0.37037037037037035, 0.29411764705882354, 0.3...","[0.3663003663003663, 0.2976190476190476, 0.369...","[0.3846153846153846, 0.2857142857142857, 0.370...","[0.37453183520599254, 0.29325513196480935, 0.3...",3,1.028537,1.010086,1.010729,1.029341,1.033145,1.025923
4,17/08/2024,15:00,Newcastle,Southampton,1,0,"[1, 0, 0]","[0.7352941176470588, 0.19047619047619047, 0.125]","[0.7407407407407407, 0.18181818181818182, 0.12...","[0.7518796992481203, 0.18181818181818182, 0.11...","[0.7407407407407407, 0.17543859649122806, 0.12...","[0.7407407407407407, 0.18181818181818182, 0.125]","[0.7407407407407407, 0.17793594306049823, 0.12...",1,0.357008,0.350409,0.335249,0.336814,0.346567,0.341375
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
533,19/05/2024,16:00,Crystal Palace,Aston Villa,5,0,"[1, 0, 0]","[0.5405405405405405, 0.23094688221709006, 0.27...","[0.3484320557491289, 0.25641025641025644, 0.45...","[nan, nan, nan]","[0.5464480874316939, 0.22883295194508008, 0.26...","[0.5405405405405405, 0.23809523809523808, 0.28...","[0.53475935828877, 0.2336448598130841, 0.27247...",5,0.663276,0.846149,NaN,0.646092,0.677550,0.666009
534,19/05/2024,16:00,Liverpool,Wolves,2,0,"[1, 0, 0]","[0.8547008547008548, 0.125, 0.06666666666666667]","[0.9009009009009008, 0.08695652173913043, 0.05...","[nan, nan, nan]","[0.8620689655172414, 0.10787486515641856, 0.07...","[0.8771929824561404, 0.11764705882352941, 0.06...","[0.8620689655172414, 0.11481056257175659, 0.06...",2,0.202328,0.142753,NaN,0.191183,0.190718,0.193019
535,19/05/2024,16:00,Luton,Fulham,2,4,"[0, 0, 1]","[0.3448275862068966, 0.25641025641025644, 0.45...","[0.13793103448275862, 0.16666666666666666, 0.7...","[nan, nan, nan]","[0.341296928327645, 0.25510204081632654, 0.442...","[0.35714285714285715, 0.25, 0.45454545454545453]","[0.3424657534246575, 0.2531645569620253, 0.448...",6,0.842740,0.340119,NaN,0.853505,0.848318,0.845119
536,19/05/2024,16:00,Man City,West Ham,3,1,"[1, 0, 0]","[0.9259259259259258, 0.08333333333333333, 0.04...","[1.11, 11.5, 19.5]","[nan, nan, nan]","[0.9174311926605504, 0.07627765064836003, 0.04...","[0.9345794392523364, 0.08333333333333333, 0.04...","[0.9090909090909091, 0.08532423208191126, 0.04...",4,0.132281,NaN,NaN,0.128042,0.131133,0.134786


In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

for col in bet_data.columns[14:]:
  print(col,': ',bet_data[col].mean())

0.5224316
0.33251783
0.2025174
1.025923
0.3413746
1.2898116
1.0645522
0.94133145
0.6467638
1.5158384
0.956054
1.2217181
0.6462782
0.12219971
1.0853513
0.4082894
0.6193444
1.3761915
0.6138519
0.26613086
1.7733402
0.61123455
0.9908116
1.2931323
0.59302807
1.3364987
0.3602339
1.5211813
0.9603881
0.6548062
0.59458125
1.6851246
1.4607973
1.2891768
2.5593803
0.20031737
0.45439747
0.7576665
0.88718694
0.78836274
0.7384268
0.5064459
1.0926484
1.276741
0.29446298
1.3015136
0.47927076
1.3562304
1.3784837
1.3294666
1.5378538
0.20614995
1.32678
0.594071
1.0449768
1.1626283
0.3442598
1.367233
1.059806
0.5427105
0.44128698
0.16983108
0.7777095
1.2173826
0.25668126
0.63024724
1.3277444
1.3804957
1.5982482
1.1064616
0.44934103
1.1019868
1.1424006
0.5773037
1.2949759
1.1494199
1.6386666
0.31587166
0.5333491
0.9049602
0.9186235
1.4193443
0.55084723
1.5059956
0.14342412
1.2576563
0.6082654
1.4221311
1.1050314
1.2583199
1.3675499
1.7127361
1.3240447
0.3733771
0.80541176
1.0144097
1.2632381
0.70203924
1.37